# Day Multi-Turn Integration Test

낮 파이프라인을 여러 턴에 걸쳐 자동으로 실행합니다.

```
DayController.process()   ← tool_call (LLM) → tool 실행 → rule_engine
  ↓
wsm.apply_delta()         ← 상태 업데이트
  ↓
EndingChecker.check()     ← 엔딩 체크
  ↓
NarrativeLayer.render()   ← 나레이션 생성 (LLM)
```

각 단계를 개별적으로 try/except해서 **어느 단계에서 연결이 끊어졌는지** 확인합니다.

In [10]:
# ── 0. 환경 설정 ────────────────────────────────────────────
import sys, logging, traceback
from pathlib import Path
from dotenv import load_dotenv

ROOT = Path.cwd()
while not (ROOT / "app").exists() and ROOT != ROOT.parent:
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

load_dotenv(ROOT / ".env")

# 로그: WARNING 이상만 출력 (INFO 억제), 에러는 명시적으로 출력
logging.basicConfig(level=logging.WARNING, format="%(name)s | %(levelname)s | %(message)s")

print(f"Project root : {ROOT}")

Project root : c:\Users\amate\GIT\game_demo


In [ ]:
# ── 0-1. LoRA / PostProcess 로그 캡처 설정 ──────────────────────────────────
import logging


class LogCapture(logging.Handler):
    """특정 로거의 레코드를 버퍼에 캡처하는 핸들러"""

    def __init__(self):
        super().__init__(level=logging.DEBUG)
        self.records: list[logging.LogRecord] = []

    def emit(self, record: logging.LogRecord) -> None:
        self.records.append(record)

    def clear(self) -> None:
        self.records.clear()


_log_capture = LogCapture()

# 관심 로거에 핸들러 부착 (DEBUG 레벨 활성화)
_CAPTURE_LOGGERS = ["app.tools", "app.llm.engine", "app.postprocess"]
for _name in _CAPTURE_LOGGERS:
    _lg = logging.getLogger(_name)
    _lg.setLevel(logging.DEBUG)
    if _log_capture not in _lg.handlers:
        _lg.addHandler(_log_capture)

print("LogCapture 설정 완료:", _CAPTURE_LOGGERS)

In [11]:
# ── 1. 시나리오 로드 & WorldState 초기화 ────────────────────
from app.loader import ScenarioLoader
from app.state import get_world_state_manager

SCENARIO_ID = "coraline"
USER_ID     = "test_runner"

loader = ScenarioLoader(ROOT / "scenarios")
assets = loader.load(SCENARIO_ID)

wsm   = get_world_state_manager()
wsm.reset_state(USER_ID, SCENARIO_ID)            # 항상 깨끗하게 시작
world = wsm.get_state(USER_ID, SCENARIO_ID, assets)

print(f"시나리오  : {assets.scenario.get('title')}")
print(f"NPC       : {list(world.npcs.keys())}")
print(f"초기 vars : {world.vars}")
print(f"초기 인벤 : {world.inventory}")
print(f"턴 제한   : {assets.get_turn_limit()}")

시나리오  : 인형의 집 (Puppet Home)
NPC       : ['stepmother', 'stepfather', 'brother', 'grandmother', 'dog_baron']
초기 vars : {'humanity': 100, 'suspicion_level': 0, 'day': 1, 'house_on_fire': False}
초기 인벤 : ['warm_black_tea']
턴 제한   : 50


In [12]:
# ── 2. 컴포넌트 임포트 ──────────────────────────────────────
from app.day_controller import get_day_controller
from app.ending_checker import EndingChecker
from app.narrative import get_narrative_layer

controller      = get_day_controller()
ending_checker  = EndingChecker()
narrative       = get_narrative_layer()

print("컴포넌트 초기화 완료")

컴포넌트 초기화 완료


In [ ]:
# ── 4. 단계별 헬퍼 ──────────────────────────────────────────
OK   = "✅"
FAIL = "❌"

def status(label: str, ok: bool, detail: str = "") -> str:
    icon = OK if ok else FAIL
    line = f"  {icon} {label}"
    if detail:
        line += f" | {detail}"
    return line


def _print_llm_logs() -> None:
    """캡처된 LoRA / PostProcess 로그를 포매팅해서 출력 후 버퍼 클리어"""
    records = list(_log_capture.records)
    _log_capture.clear()
    for rec in records:
        msg = rec.getMessage()
        name = rec.name
        if name == "app.llm.engine":
            if "LoRA 어댑터 사용" in msg:
                print(f"      🔧 {msg}")
            elif "[LoRA 출력]" in msg:
                print(f"      📝 {msg}")
        elif name == "app.tools" and "[LoRA raw]" in msg:
            raw = msg.split("[LoRA raw]", 1)[-1].strip()
            print(f"      📝 LoRA raw    | {raw}")
        elif name == "app.postprocess":
            print(f"      🔄 PostProcess | {msg}")


def run_turn(user_input: str, world_state, turn_no: int):
    """
    한 턴을 실행하고 각 단계의 성공/실패를 출력합니다.
    LoRA 원문과 PostProcess 결과를 추가로 표시합니다.

    Returns: (updated_world_state, ended: bool)
    """
    print(f"\n{'═' * 60}")
    print(f"  TURN {turn_no}  |  {user_input}")
    print(f"{'═' * 60}")

    tool_result  = None
    world_after  = world_state
    ended        = False
    narrative_ok = False

    _log_capture.clear()  # 이전 로그 초기화

    # ── Step 1: DayController.process() ─────────────────────
    try:
        tool_result = controller.process(user_input, world_state, assets)
        log = controller.decision_log[-1]
        sel = log["tool_selection"]
        detail = f"tool={sel['tool_name']}  intent={log['intent']}  args={sel['args']}"
        print(status("DayController.process", True, detail))
        _print_llm_logs()   # ← LoRA / PostProcess 로그 출력
        if tool_result.npc_response:
            print(f"      NPC 응답: {tool_result.npc_response}")
        if tool_result.event_description:
            for ev in tool_result.event_description:
                print(f"      사건: {ev}")
    except Exception as e:
        print(status("DayController.process", False, str(e)))
        traceback.print_exc()
        _log_capture.clear()
        # 이후 단계는 tool_result 없이도 부분적으로 진행

    # ── Step 2: wsm.apply_delta() ───────────────────────────
    try:
        delta = tool_result.state_delta if tool_result else {}
        # turn 증가를 delta에 반영 (DayController는 turn을 직접 올리지 않음)
        delta["turn_increment"] = delta.get("turn_increment", 1)
        wsm.persist(USER_ID, SCENARIO_ID, world_state)   # 현재 상태를 store에 반영
        world_after = wsm.apply_delta(USER_ID, SCENARIO_ID, delta, assets)
        print(status("wsm.apply_delta", True,
                     f"turn {world_state.turn}→{world_after.turn}  "
                     f"vars={world_after.vars}"))
    except Exception as e:
        print(status("wsm.apply_delta", False, str(e)))
        traceback.print_exc()

    # ── Step 3: EndingChecker.check() ───────────────────────
    try:
        ending_result = ending_checker.check(world_after, assets)
        if ending_result.reached:
            detail = f"🎬 {ending_result.ending.name}  ({ending_result.ending.ending_id})"
            ended = True
        else:
            detail = "미도달"
        print(status("EndingChecker.check", True, detail))
    except Exception as e:
        print(status("EndingChecker.check", False, str(e)))
        traceback.print_exc()
        ending_result = None

    # ── Step 4: NarrativeLayer.render() ─────────────────────
    try:
        if ended and ending_result:
            ending_info = {
                "ending_id":       ending_result.ending.ending_id,
                "name":            ending_result.ending.name,
                "epilogue_prompt": ending_result.ending.epilogue_prompt,
            }
            narrative_text = narrative.render_ending(ending_info, world_after, assets)
            mode = "ending"
        else:
            ev    = tool_result.event_description if tool_result else []
            delta = tool_result.state_delta if tool_result else {}
            npc_r = tool_result.npc_response if tool_result else None
            narrative_text = narrative.render(
                world_state=world_after,
                assets=assets,
                event_description=ev,
                state_delta=delta,
                npc_response=npc_r,
            )
            mode = "day"
        print(status("NarrativeLayer.render", True, f"mode={mode}  len={len(narrative_text)}자"))
        narrative_ok = True
    except Exception as e:
        print(status("NarrativeLayer.render", False, str(e)))
        traceback.print_exc()
        narrative_text = "(나레이션 생성 실패)"

    # ── 나레이션 출력 ────────────────────────────────────────
    print(f"\n{'─' * 50}")
    print(narrative_text)
    print(f"{'─' * 50}")

    return world_after, ended

In [14]:
# ── 4. 단계별 헬퍼 ──────────────────────────────────────────
OK   = "✅"
FAIL = "❌"

def status(label: str, ok: bool, detail: str = "") -> str:
    icon = OK if ok else FAIL
    line = f"  {icon} {label}"
    if detail:
        line += f" | {detail}"
    return line


def run_turn(user_input: str, world_state, turn_no: int):
    """
    한 턴을 실행하고 각 단계의 성공/실패를 출력합니다.
    어느 단계에서 실패했어도 다음 단계는 최대한 시도합니다.

    Returns: (updated_world_state, ended: bool)
    """
    print(f"\n{'═' * 60}")
    print(f"  TURN {turn_no}  |  {user_input}")
    print(f"{'═' * 60}")

    tool_result  = None
    world_after  = world_state
    ended        = False
    narrative_ok = False

    # ── Step 1: DayController.process() ─────────────────────
    try:
        tool_result = controller.process(user_input, world_state, assets)
        log = controller.decision_log[-1]
        sel = log["tool_selection"]
        detail = f"tool={sel['tool_name']}  intent={log['intent']}  args={sel['args']}"
        print(status("DayController.process", True, detail))
        if tool_result.npc_response:
            print(f"      NPC 응답: {tool_result.npc_response}")
        if tool_result.event_description:
            for ev in tool_result.event_description:
                print(f"      사건: {ev}")
    except Exception as e:
        print(status("DayController.process", False, str(e)))
        traceback.print_exc()
        # 이후 단계는 tool_result 없이도 부분적으로 진행

    # ── Step 2: wsm.apply_delta() ───────────────────────────
    try:
        delta = tool_result.state_delta if tool_result else {}
        # turn 증가를 delta에 반영 (DayController는 turn을 직접 올리지 않음)
        delta["turn_increment"] = delta.get("turn_increment", 1)
        wsm.persist(USER_ID, SCENARIO_ID, world_state)   # 현재 상태를 store에 반영
        world_after = wsm.apply_delta(USER_ID, SCENARIO_ID, delta, assets)
        print(status("wsm.apply_delta", True,
                     f"turn {world_state.turn}→{world_after.turn}  "
                     f"vars={world_after.vars}"))
    except Exception as e:
        print(status("wsm.apply_delta", False, str(e)))
        traceback.print_exc()

    # ── Step 3: EndingChecker.check() ───────────────────────
    try:
        ending_result = ending_checker.check(world_after, assets)
        if ending_result.reached:
            detail = f"🎬 {ending_result.ending.name}  ({ending_result.ending.ending_id})"
            ended = True
        else:
            detail = "미도달"
        print(status("EndingChecker.check", True, detail))
    except Exception as e:
        print(status("EndingChecker.check", False, str(e)))
        traceback.print_exc()
        ending_result = None

    # ── Step 4: NarrativeLayer.render() ─────────────────────
    try:
        if ended and ending_result:
            ending_info = {
                "ending_id":       ending_result.ending.ending_id,
                "name":            ending_result.ending.name,
                "epilogue_prompt": ending_result.ending.epilogue_prompt,
            }
            narrative_text = narrative.render_ending(ending_info, world_after, assets)
            mode = "ending"
        else:
            ev    = tool_result.event_description if tool_result else []
            delta = tool_result.state_delta if tool_result else {}
            npc_r = tool_result.npc_response if tool_result else None
            narrative_text = narrative.render(
                world_state=world_after,
                assets=assets,
                event_description=ev,
                state_delta=delta,
                npc_response=npc_r,
            )
            mode = "day"
        print(status("NarrativeLayer.render", True, f"mode={mode}  len={len(narrative_text)}자"))
        narrative_ok = True
    except Exception as e:
        print(status("NarrativeLayer.render", False, str(e)))
        traceback.print_exc()
        narrative_text = "(나레이션 생성 실패)"

    # ── 나레이션 출력 ────────────────────────────────────────
    print(f"\n{'─' * 50}")
    print(narrative_text)
    print(f"{'─' * 50}")

    return world_after, ended

In [15]:
# ── 5. 멀티턴 실행 ───────────────────────────────────────────
current_world = world   # 초기 상태

for turn_no, user_input in enumerate(TEST_INPUTS, start=1):
    current_world, game_over = run_turn(user_input, current_world, turn_no)

    if game_over:
        print(f"\n{'=' * 60}")
        print("  게임 종료 (엔딩 도달)")
        print(f"{'=' * 60}")
        break

print(f"\n{'=' * 60}")
print("멀티턴 테스트 완료")
print(f"{'=' * 60}")


════════════════════════════════════════════════════════════
  TURN 1  |  엄마에게 오늘은 무엇을 할 거냐고 물어본다
════════════════════════════════════════════════════════════
  ✅ DayController.process | tool=interact  intent=neutral  args={'target': 'stepmother', 'interact': '엄마에게 오늘은 무엇을 할 거냐고 물어본다'}
      NPC 응답: 엄마, 오늘은! 무엇을! 할 거야?!! [triggers: plus] [taboos: -]

["새엄마 (엘리노어) 대사"]
"오늘은 특별한 일이 있네! 네가 준비하는 건 어떨까?! 어떨까?! 엄마도 도와줄게!
      사건: 새엄마 (엘리노어)는 플레이어에게 특별한 일을 제안하며 긍정적인 반응을 보여 플레이어와 새엄마 모두 plus_hits가 증가했습니다.
  ✅ wsm.apply_delta | turn 1→2  vars={'humanity': 100, 'suspicion_level': 0, 'day': 1, 'house_on_fire': False}
  ✅ EndingChecker.check | 미도달
  ✅ NarrativeLayer.render | mode=day  len=910자

──────────────────────────────────────────────────
"

새엄마의 목소리가 귓가에 부드럽게 울렸다. 그녀의 눈빛은 기대와 함께 어딘가 이상한 빛을 띠고 있었다. "오늘은 특별한 일이 있네! 네가 준비하는 건 어떨까?! 어떨까?!" 그녀의 손길은 부드럽게 플레이어의 어깨를 두드리며, 무언가 숨겨진 계획을 펼치려는 듯했다. 엘리노어는 항상 그렇게, 플레이어에게는 친근하게 다가와도, 어떤 이유에서인지 그 안에는 항상 미지의 세계가 감돌았다. "엄마도 도와줄게!" 그 말이 떨어지는 순간, 플레이어의 plus_h

In [16]:
# ── 6. 최종 상태 요약 ────────────────────────────────────────
print("=" * 60)
print("최종 WorldState")
print("=" * 60)
print(f"  턴        : {current_world.turn}")
print(f"  vars      : {current_world.vars}")
print(f"  inventory : {current_world.inventory}")
print(f"  flags     : {dict(current_world.flags)}")
print()
print("NPC 스탯:")
for npc_id, npc_state in current_world.npcs.items():
    npc_data = assets.get_npc_by_id(npc_id) or {}
    name = npc_data.get("name", npc_id)
    print(f"  [{name}] {npc_state.stats}")

print()
print("의사결정 로그:")
for log in controller.decision_log:
    sel = log["tool_selection"]
    print(f"  Turn {log['turn']:2d} | {sel['tool_name']:10s} | intent={log['intent']:12s} | {log['user_input'][:40]}")

최종 WorldState
  턴        : 6
  vars      : {'humanity': 100, 'suspicion_level': 4, 'day': 1, 'house_on_fire': False}
  inventory : ['warm_black_tea']
  flags     : {'ending': None, 'basement_unlocked': False, 'truth_revealed': False, 'status_effects': []}

NPC 스탯:
  [새엄마 (엘리노어)] {'affection': 48, 'fear': 80, 'humanity': 0, 'plus_hits': 0, 'minus_hits': 0, 'critical_hits': 0}
  [새아빠 (아더)] {'affection': 29, 'fear': 60, 'humanity': 20}
  [동생 (루카스)] {'affection': 60, 'fear': 40, 'humanity': 50}
  [할머니 (마가렛)] {'affection': 20, 'fear': 10, 'humanity': 10}
  [강아지 (바론)] {'affection': 70, 'fear': 0, 'humanity': 100}

의사결정 로그:
  Turn  1 | interact   | intent=neutral      | 엄마에게 오늘은 무엇을 할 거냐고 물어본다
  Turn  2 | action     | intent=investigate  | 거실을 조심스럽게 둘러본다
  Turn  3 | interact   | intent=investigate  | 딸에게 이 집에서 가장 무서운 게 뭔지 물어본다
  Turn  4 | action     | intent=investigate  | 창문 밖을 바라보며 탈출 경로를 상상한다
  Turn  5 | interact   | intent=obey         | 아빠에게 밖에 나가도 되냐고 부탁한다
  Turn  1 | interact   | inten

## 6. 인터랙티브 모드

직접 입력해서 대화를 진행합니다. (최대 10턴)

- 각 턴마다 위와 동일한 파이프라인 단계 리포트가 출력됩니다.
- 엔딩 도달 시 자동 종료됩니다.
- **종료**: `q` 또는 `quit` 입력

먼저 **6-1 셀**로 WorldState를 초기화한 뒤, **6-2 셀**을 실행하세요.

In [17]:
import logging
logging.basicConfig(level=logging.DEBUG)
# ── 6-1. 인터랙티브 WorldState 초기화 ───────────────────────
# (자동 테스트와 독립적으로 초기 상태에서 시작)
wsm.reset_state(USER_ID, SCENARIO_ID)
_iworld = wsm.get_state(USER_ID, SCENARIO_ID, assets)
_iturn  = 1

print(f"시나리오  : {assets.scenario.get('title')}")
print(f"NPC       : {list(_iworld.npcs.keys())}")
print(f"초기 vars : {_iworld.vars}")
print(f"턴 제한   : {assets.get_turn_limit()}")
print()
print("초기화 완료 — 아래 셀을 실행해 인터랙티브 모드 시작")

시나리오  : 인형의 집 (Puppet Home)
NPC       : ['stepmother', 'stepfather', 'brother', 'grandmother', 'dog_baron']
초기 vars : {'humanity': 100, 'suspicion_level': 0, 'day': 1, 'house_on_fire': False}
턴 제한   : 50

초기화 완료 — 아래 셀을 실행해 인터랙티브 모드 시작


In [ ]:
# ── 6-2. 인터랙티브 루프 (최대 10턴) ───────────────────────
MAX_TURNS = 10

print("=" * 60)
print(f"  인터랙티브 모드  |  Turn {_iturn}  |  최대 {MAX_TURNS}턴")
print(f"  종료: q 또는 quit")
print("=" * 60)

while _iturn <= MAX_TURNS:
    # ── 입력 ─────────────────────────────────────────────────
    try:
        text = input(f"\n[Turn {_iturn}/{MAX_TURNS}] 입력 > ").strip()
    except (EOFError, KeyboardInterrupt):
        print("\n종료합니다.")
        break

    if not text:
        continue
    if text.lower() in ("q", "quit", "exit"):
        print("종료합니다.")
        break

    # ── 파이프라인 실행 ───────────────────────────────────────
    _iworld, game_over = run_turn(text, _iworld, _iturn)

    # ── 상태 요약 ─────────────────────────────────────────────
    print(f"\n  [현재 상태]")
    print(f"    Turn {_iturn} → {_iworld.turn}  |  vars: {_iworld.vars}")
    for npc_id, npc_state in _iworld.npcs.items():
        npc_data = assets.get_npc_by_id(npc_id) or {}
        name = npc_data.get("name", npc_id)
        print(f"    [{name}] {npc_state.stats}")
    if _iworld.inventory:
        print(f"    inventory: {_iworld.inventory}")

    _iturn += 1

    if game_over:
        print(f"\n{'=' * 60}")
        print("  게임 종료 (엔딩 도달)")
        print(f"{'=' * 60}")
        break
    if _iturn > MAX_TURNS:
        print(f"\n{'=' * 60}")
        print(f"  {MAX_TURNS}턴 완료 — 루프 종료")
        print(f"{'=' * 60}")

  인터랙티브 모드  |  Turn 1  |  최대 10턴
  종료: q 또는 quit

════════════════════════════════════════════════════════════
  TURN 1  |  엄마 나 나가고 싶어
════════════════════════════════════════════════════════════
  ✅ DayController.process | tool=interact  intent=rebel  args={'target': 'stepmother', 'interact': '엄마 나 나가고 싶어'}
      NPC 응답: "나가고 싶어?! 싶어?!?! 하지만! 엄마가! 네 곁에서 보호하고 싶어!!
      사건: 플레이어는 나가고 싶다는 욕구를 표현했지만, 새엄마는 엄격하게 반응해 플레이어와 새엄마 간의 긴장감이 증가했습니다.
  ✅ wsm.apply_delta | turn 1→2  vars={'humanity': 100, 'suspicion_level': 5, 'day': 1, 'house_on_fire': False}
  ✅ EndingChecker.check | 미도달
  ✅ NarrativeLayer.render | mode=day  len=696자

──────────────────────────────────────────────────
"나가고 싶어?! 싶어?!?! 하지만! 엄마가! 네 곁에서 보호하고 싶어!!" 새엄마의 목소리는 거친 돌에 맞추어진 것처럼 깊게 울렸다. 플레이어는 순간적으로 무릎을 꿇어 어두운 코너로 몸을 숨겼다. 새엄마의 시선이 그 방향으로 향하는 것을 느끼자, 긴장감은 무게를 더해 찢어질 듯한 공포 속에서 플레이어를 감쌌다. 그녀의 말이 여전히 귓가에서 울려 퍼졌지만, 이제는 그녀의 목소리보다 더 큰 것은 플레이어의 가슴속에서 뛰어대는 두려움이었다. "나가고 싶어, 하지만…" 플레이어는 눈을 감아야 했다. "하지만!" 새엄마의 목소리가 마지막으로 울려 퍼져, 이내 침묵이 